# Accessibilité relative des résidus et évolution protéique

Il y a plusieurs étapes dans le traitement des données :

 * obtenir les données pdb par **PISCES**
 * faire tourner **DSSP** pour obtenir l'accessibilité du solvant
 * récupération des séquences homologues par **PSI-BLAST** (pas moins de 70% d'identité)
 * réalignement avec la séquence parente
 * repositionnement des chaines latérales avec **SCWRL4** -> accessibilité du solvant ? 

## PISCES

Sélection de la base de données la plus petite base de [données](http://dunbrack.fccc.edu/Guoli/culledpdb_hh/cullpdb_pc20_res1.6_R0.25_d201103_chains3760.gz)

*Afin de simplifier le traitement on travaille tout d'abord avec un fichier PDB, on résoud les problèmes, une fois que les exceptions sont réglées, on automatise.*